In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from decimal import Decimal

# Configuration
PRECISION = Decimal("1e18")  # 10^18 precision as used in Vyper
NUM_STEPS = 3_00


# Function to update xcp_profit as in the Curve contract
def update_xcp_profit(old_xcp_profit, virtual_price, old_virtual_price):
    return old_xcp_profit * virtual_price / old_virtual_price


# Generate a virtual price series with random fluctuations
def main():
    # Initialize both metrics at 10^18 (1.0)
    virtual_prices = [PRECISION]
    xcp_profits = [PRECISION]
    xcp_profit_a = PRECISION
    # Generate random virtual price changes
    for i in range(1, NUM_STEPS):
        # Admin fee claim (LP+DAO)
        fee_claim_flag = bool(i % 15 == 0)
        if fee_claim_flag:
            crude_profit = xcp_profits[-1] - xcp_profit_a
            if crude_profit > 0:
                fee_amount = Decimal(int(crude_profit * Decimal(0.01) / 2))
                xcp_profits[-1] -= fee_amount
                xcp_profit_a = xcp_profits[-1]
                virtual_prices[-1] -= fee_amount
                print(f"fee claim {i}, {fee_amount}")
            # skip rest of the loop
            continue

        # Random change between 0% and +0.15% (slightly biased toward growth)
        # emulate swap -> fees boost virtual price
        change_pct = Decimal(np.random.uniform(0.00, 0.0015))
        # Calculate new virtual price (simplified from contract)
        new_vp = virtual_prices[-1] * (Decimal("1") + change_pct)

        # Ensure integer arithmetic like in contract
        new_vp = Decimal(int(new_vp))

        # Calculate new xcp_profit
        new_xcp = update_xcp_profit(xcp_profits[-1], new_vp, virtual_prices[-1])

        # emulate tweak_price
        thr = Decimal(2e12)

        # Rebalance fee spend
        rebalance_flag = bool(i % 10 == 0)
        rebalance_buffer = PRECISION + (new_xcp - PRECISION) / 2
        if rebalance_flag and new_vp > rebalance_buffer + thr:
            change_pct = Decimal(np.random.uniform(-0.005, -0.001))
            # Calculate new virtual price (simplified from contract)
            # candidate_vp = virtual_prices[-1] * (Decimal('1') + change_pct)
            candidate_vp = rebalance_buffer + thr
            candidate_vp = Decimal(int(candidate_vp))
            if candidate_vp >= max(rebalance_buffer, PRECISION):
                new_vp = candidate_vp
                print(f"rebalance {i}, {new_vp}")
        virtual_prices.append(new_vp)
        xcp_profits.append(new_xcp)

    # Convert to regular floats for plotting (normalize to 1.0)
    vp_display = [float(v / PRECISION) for v in virtual_prices]
    xcp_display = [float(x / PRECISION) for x in xcp_profits]
    xcp_half = [float((PRECISION + (x - PRECISION) / 2) / PRECISION) for x in xcp_profits]
    xcp_sqrt = [float(np.sqrt(x / PRECISION)) for x in xcp_profits]
    # Calculate the ratio between them
    ratios = [xcp / vp for xcp, vp in zip(xcp_display, vp_display)]

    # Visualization
    plt.figure(figsize=(10, 8))

    # Plot values
    plt.subplot(2, 1, 1)
    plt.plot(vp_display, label="Virtual Price", linewidth=2)
    plt.plot(xcp_display, label="XCP Profit", linewidth=2, linestyle="--")
    plt.plot(xcp_half, label="XCP Half", linewidth=2, linestyle=":")
    plt.plot(xcp_sqrt, label="XCP Sqrt", linewidth=2, linestyle=":")
    plt.title("Virtual Price and XCP Profit Over Time")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)

    # Plot ratio
    plt.subplot(2, 1, 2)
    plt.plot(ratios, label="XCP Profit / Virtual Price")
    plt.axhline(y=1.0, color="r", linestyle="--")
    plt.title("Ratio of XCP Profit to Virtual Price")
    plt.ylabel("Ratio")
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    # Print basic statistics
    print(f"Final VP: {vp_display[-1]:.6f}, Final XCP: {xcp_display[-1]:.6f}")
    print(f"Are VP and XCP proportional? {all(abs(r - 1.0) < 1e-10 for r in ratios)}")
    print(f"Max deviation from 1.0: {max(abs(r - 1.0) for r in ratios):.10f}")


if __name__ == "__main__":
    main()

In [ ]:
r = 0.0015
n = 3000

init = 1

res = (1 + r / 2) ** n

print(res)

In [ ]:
wad = 10**18
pool_value_init = 3000000000000000000000000
pool_value_post = 6019243555149172634124923
total_fees = pool_value_post - pool_value_init
vp_0 = 1000000000000000000
vp_1 = 2006414292517988114
xcp_profit_0 = 1000000000000000000
xcp_profit_1 = 2006414292517988114
lp_supply = 38729833462074168852585
D_0 = 3000000000000000000000000
D_1 = 6009324343410821147499040
# fees: uint256 = unsafe_div(
#     unsafe_sub(xcp_profit, xcp_profit_a) * ADMIN_FEE, 2 * 10**10
# )

fees = (xcp_profit_1 - xcp_profit_0) // 2 // 2 * 2
print(f"fees: {fees}")
frac = vp_1 * 10**18 // (vp_1 - fees) - 10**18
print(f"frac: {frac}")
admin_share = lp_supply * frac // 10**18
print(f"admin_share: {admin_share}")
fees_rate = admin_share * 10**18 // (admin_share + lp_supply)
fees_value = fees_rate * pool_value_post // 10**18
print(f"fees value: {fees_value}")
print(f"pool_profits: {(pool_value_post - pool_value_init)}")
print(f"rate: {fees_value / (pool_value_post - pool_value_init)}")

In [ ]:
vp_1 / vp_0

In [ ]:
pool_value_post / pool_value_init

In [ ]:
pool_value_init

In [ ]:
pool_value_post

In [ ]:
D_1 / D_0